# Lab 10.1: CNN with MNIST Classifier

Edited By Steve Ive

Here, we are going to build simple CNN model for MNIST Classifying. To more understand about CNN, please refer before script "10.0 - About CNN".

Reference from

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_1_mnist_cnn.ipynb

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed(1)

In [3]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download = True,
                             train = True,
                             transform = transforms.ToTensor())
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download = True,
                            train = False,
                            transform = transforms.ToTensor())

C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
batch_size = 100
training_epochs = 20
learning_rate = 0.001

In [5]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

In [6]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Layer 1: ImageInput Shape=(?, 28, 28, 1)
        # Conv   : ImageInput Shape=(?, 28, 28, 32)
        # Pool   : ImageInput Shape=(?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # Layer 2: ImageInput Shape=(?, 14, 14, 32)
        # Conv   : ImageInput Shape=(?, 14, 14, 64)
        # Pool   : ImageInput Shape=(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        # Final Fully Connected
        # 7 x 7 x 64 inputs => 10 outputs
        self.fc = nn.Linear(7 * 7 * 64, 10)
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.size()) #torch.size([100, 64, 7, 7])
        out = out.view(out.size(0), -1) # Flatten for FC
        out = self.fc(out)
        return out    

In [7]:
model = MNIST_CNN().to(device)

In [8]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
total_batch = len(data_loader)

print('Learning started')

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28 x 28), no reshape
        # Label is not one-hot encoded

        X = X.to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y).to(device)

        #Reduce the cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {} / {}, cost: {:.6f}'.format(epoch + 1, training_epochs, avg_cost))
print('Learning Finished')

Learning started


C:\Users\buddhalight\envs\buddhalight\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1 / 20, cost: 0.218919
Epoch: 2 / 20, cost: 0.059666
Epoch: 3 / 20, cost: 0.044729
Epoch: 4 / 20, cost: 0.034445
Epoch: 5 / 20, cost: 0.029801
Epoch: 6 / 20, cost: 0.024727
Epoch: 7 / 20, cost: 0.020556
Epoch: 8 / 20, cost: 0.017596
Epoch: 9 / 20, cost: 0.014956
Epoch: 10 / 20, cost: 0.012632
Epoch: 11 / 20, cost: 0.009928
Epoch: 12 / 20, cost: 0.009620
Epoch: 13 / 20, cost: 0.007476
Epoch: 14 / 20, cost: 0.007555
Epoch: 15 / 20, cost: 0.006531
Epoch: 16 / 20, cost: 0.005704
Epoch: 17 / 20, cost: 0.003859
Epoch: 18 / 20, cost: 0.003481
Epoch: 19 / 20, cost: 0.004580
Epoch: 20 / 20, cost: 0.004117
Learning Finished


### Take a Moment!

Why we should ```.view(len(mnist_test), 1, 28 ,28)``` below despite of we didn't do that upon?

=> since ```data_loader``` makes mini batches as size of size([100, 1, 28, 28]), we don't have to do that manually, but we don't use ```data_loader``` below.

In [10]:
#Test model and check accuracy
with torch.no_grad():
    #print(mnist_test.data.shape) => torch.Size([10000, 28, 28])
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    #prediction
    pred = model(X_test)
    correct_prediction = (torch.argmax(pred, 1) == Y_test)
    accuracy = correct_prediction.float().mean()

    print('Accuracy: ', accuracy.item())

Accuracy:  0.9842999577522278
